In [1]:
import pandas
import os
from gene_panel_selection import ExpressionDataset, PROPOSEMethod, SeuratMethod

In [2]:
import torch.cuda
print("cuda devices:")
for i in range(torch.cuda.device_count()):
    print(f"  {i}: {torch.cuda.get_device_name(i)}")    

cuda devices:
  0: Quadro P1000


In [22]:
data_path = '../../../data/macaque_nac/'
expression = pandas.read_feather(data_path + 'counts.feather')
annotation = pandas.read_feather(data_path + 'anno.feather').set_index('sample_id')[['class_label', 'subclass_label', 'supertype_label']]

In [23]:
expression

,U6,ZNF692,ZNF672,SH3BP5L,OR2T27,OR2T10,OR2T1,OR2T6,OR2M4,Y-RNA ENSMMUG00000062908,...,PGBD2,5S-rRNA ENSMMUG00000053442,Metazoa-SRP ENSMMUG00000053951,U3 ENSMMUG00000064655,U3 ENSMMUG00000061317,U3 ENSMMUG00000035418,Metazoa-SRP ENSMMUG00000053315,U4 ENSMMUG00000064166,Metazoa-SRP ENSMMUG00000054492,Metazoa-SRP ENSMMUG00000056295
sample_id,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAGCCTGCA-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAGGCTTCG-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCATGGCCCA-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCATTAGGCC-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACATGCAACTAACT-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCGTGCTTC-L8XR_220210_02_C04-1160243210-1-0-0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTAAGCTTG-L8XR_220210_02_C04-1160243210-1-0-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTCCGTGAG-L8XR_220210_02_C04-1160243210-1-0-0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
annotation


,class_label,subclass_label,supertype_label
sample_id,,,
AAACAGCCAGCCTGCA-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,MSNs,D1.MSN,D1.Shell.OT
AAACAGCCAGGCTTCG-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,Interneurons,Unassigned,IN.Unknown_1
AAACAGCCATGGCCCA-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,Endothelial,Endothelial,Endothelial
AAACAGCCATTAGGCC-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,MSNs,D1.MSN,D1.D2.GRIA4.HTR2C
AAACATGCAACTAACT-L8XR_220210_02_A04-1160243780-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,Interneurons,Unassigned,IN.Unknown_1
...,...,...,...
TTTGTGTTCGTGCTTC-L8XR_220210_02_C04-1160243210-1-0-0,Interneurons,Unassigned,IN.Unknown_1
TTTGTTGGTAAGCTTG-L8XR_220210_02_C04-1160243210-1-0-0,Interneurons,IN.PVALB,Pvalb_2
TTTGTTGGTCCGTGAG-L8XR_220210_02_C04-1160243210-1-0-0,MSNs,Unassigned,IN.Unknown_1


In [25]:
exp_data = ExpressionDataset(
    expression_data=expression,
    annotation_data=annotation,
    expression_type='raw',
)
exp_data.expression_data.shape

(40438, 22242)

In [26]:
#  Shorten gene list to speed up PROPOSE 
seurat = SeuratMethod()
hv_selection = seurat.select_gene_panel(size=10000, data=exp_data, flavor='seurat_v3')
hv_data = hv_selection.expression_dataset()
hv_data.expression_data.shape

(40438, 10000)

In [27]:
propose = PROPOSEMethod()
selection = propose.select_gene_panel(size=40, data=hv_data, use_classes=True, annotation_column='supertype_label')

40438 total examples, 32350 training examples, 4045 validation examples, 4043 test examples
using CrossEntropyLoss, starting with lam = 0.0001


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.000100 yielded 180 genes
Reinitializing model for next iteration
next attempt is lam = 0.000035


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.000035 yielded 337 genes
Reinitializing model for next iteration
next attempt is lam = 0.000017


...

lam = 0.000009 yielded 942 genes
Warm starting model for next iteration
next attempt is lam = 0.000022


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.000022 yielded 423 genes
done, lam = 0.000022 yielded 423 genes


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

done, selected 40 genes


In [28]:
list(selection.gene_panel)

['CHRM3',
 'GRIK3',
 'KCNT2',
 'LHX8',
 'DRD3',
 'EPHB1',
 'KCNH8',
 'RXFP1',
 'ARHGAP24',
 'COL25A1',
 'NDST3',
 'NWD2',
 'THSD7A',
 'PTN',
 'CDK6',
 'GRIK1',
 'NPY',
 'FOXP2',
 'ARHGAP18',
 'RGS6',
 'PRKCH',
 'HTR2C',
 'ADARB2',
 'LRMDA',
 'GAD2',
 'IGF1',
 'NOS1',
 'PDZRN4',
 'LMO3',
 'GULP1',
 'SCN9A',
 'PDE1A',
 'RBMS1',
 'SOX6',
 'DRD2',
 'NFIB',
 'LHX6',
 'GPC6',
 'CNTNAP4',
 'ZNF536']

In [ ]:
# Report on what happened in the final selection process
selection.report()

In [ ]:
# Evaluation of final gene panel
#   number of DE genes per cluster
#   overall expression levels too high/low?
#   includes genes used for sanity checking
#   excludes genes known not to work (per-ST-method)
#   classification accuracy, confusion matrix
#   manifold preservation
evaluate_gene_panel(selection)